In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
from matplotlib import pyplot, figure
from xgboost import XGBClassifier, plot_importance
from pandas import read_csv
from keras.losses import mean_squared_error
from sklearn.model_selection import train_test_split
from numpy import sqrt
import bayes_opt
from bayes_opt import BayesianOptimization

In [ ]:
#Read in the data, set all NA's as 0
df = read_csv('ucsbdata.csv')
df.fillna(0, inplace=True)

# Split data into validation set (2007-2008), and training\testing set (2008-2018)
validation_min = '2007-08-30'
initial_start = '2008-08-30'
validation_full = df.loc[df.Index > validation_min]
validation = validation_full.loc[validation_full.Index < initial_start]
dataset = df.loc[df.Index > initial_start]

# Remove from consideration the predicted value (R) and the Date (Index)
x, y = dataset.drop(['R', 'Index'], axis = 1), dataset.R
a, b = validation.drop(['R', 'Index'], axis = 1), validation.R

In [ ]:
# Split training & testing data
x_train, x_test, y_train, y_test = train_test_split(x, 
                                                    y, 
                                                    test_size = 0.20, 
                                                    random_state = 123)
xgb.DMatrix(x_train, y_train)

In [ ]:
# Attempt to optimize hyper-parameters

# Build model with XGBRegressor
xgb_model = xgb.XGBRegressor(#base_score=0.5, 
                             #booster='gbtree', 
                             colsample_bylevel=1,
                             colsample_bynode=1, 
                             colsample_bytree=1, 
                             gamma=0,
                             importance_type='weight', 
                             learning_rate=0.1, 
                             max_delta_step=0,
                             #max_depth=3, 
                             min_child_weight=1, 
                             missing=None, 
                             n_estimators=100,
                             n_jobs=1, 
                             nthread=None, 
                             objective='reg:squarederror', 
                             random_state=0,
                             reg_alpha=0, 
                             reg_lambda=1, 
                             scale_pos_weight=1, 
                             seed=None,
                             silent=None, 
                             subsample=1, 
                             verbosity=1)
xgb_model.fit(x_train, y_train)

In [ ]:
# Find RMSE of prediction (try to get below 0.005)
prediction = xgb_model.predict(x_test)
rmse = sqrt(mean_squared_error(y_test, prediction))
rmse

In [ ]:
# Plot 20 most important features
ax = xgb.plot_importance(xgb_model, max_num_features=20)
fig = ax.figure
fig.set_size_inches(20, 20)

In [ ]:
# Plot predictions from validation data vs actual validation points 
validation_point = a.iloc[1:]
validation_actual = b.iloc[1:]

predicted_actual = xgb_model.predict(validation_point)
plt.figure(figsize=(15,10))
plt.plot(predicted_actual, label='prediction')
plt.plot(validation_actual.values, label='actual values')
plt.legend()

In [ ]:
#Compute R^2 for model

Below is code to use Bayesian optimization for XGBoost (Kaggle Demo)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop('fare_amount', axis=1),
                                                    df['fare_amount'], test_size=0.25)
del(df)
dtrain = xgb.DMatrix(X_train, label=y_train)
del(X_train)
dtest = xgb.DMatrix(X_test)
del(X_test)

In [ ]:
def xgb_evaluate(max_depth, gamma, colsample_bytree):
    params = {'eval_metric': 'rmse',
              'max_depth': int(max_depth),
              'subsample': 0.8,
              'eta': 0.1,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree}
    # Used around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=100, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [ ]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (3, 7), 
                                             'gamma': (0, 1),
                                             'colsample_bytree': (0.3, 0.9)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=3, n_iter=5, acq='ei')

In [ ]:
params = xgb_bo.res['max']['max_params']
params['max_depth'] = int(params['max_depth'])

In [ ]:
# Train a new model with the best parameters from the search
model2 = xgb.train(params, dtrain, num_boost_round=250)

# Predict on testing and training set
y_pred = model2.predict(dtest)
y_train_pred = model2.predict(dtrain)

# Report testing and training RMSE
print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(np.sqrt(mean_squared_error(y_train, y_train_pred)))